In [1]:
import ccxt
import time
from collections import deque
import datetime
import pandas as pd
import filter
import login
import atr

# 매게변수 ticker 추가 
def get_balance_coin(balances, ticker):
    bool = True
    while True:
        # for문 추가
        for i in range(1, 11):
            if ticker in balances[i]['currency']:
                balance = balances[i]['balance']
                avg_buy_price = balances[i]['avg_buy_price']
                value = (float(avg_buy_price), float(balance))
                bal_loc = int(i)
                bool = False
                break
        if bool == False:
            break
        for j in range(1, 11):
            if balances[j]['currency'] == '':
                value = (int(0), int(0))
                bal_loc = int(j)
                break
        break
    return value, bal_loc

# 매게변수 ticker 추가
def get_ticker_ma(volume, df, balances, df_2, ticker, atr_value):

    now_price = df.iloc[-1]['close']                     # 코인 현재가
    coin_check = get_balance_coin(balances, ticker) # 코인 보유 하고 있는지 체크
    avg_price = coin_check[0][0]                         # 매수 평균가
    balance = coin_check[0][1]                             # 코인 보유 개수
    # 코인의 지갑 위치값 추가
    bal_loc = coin_check[1]
    print("지갑 위치 = {}".format(bal_loc))
    # 손절매 가격
    sell_price = avg_price - (5 * atr_value)
    # 만약 현재가격이 평균가 보다 높아지면 손절매의 가격을 높인다.
    if now_price > avg_price:
        sell_price = now_price - (5 * atr_value)
    # 매수 평균가가 int 이면 매수 조건 체크 float이면 매도 조건 체크
    if type(avg_price) == int:

        if df_2.iloc[-1]['MA10'] > df_2.iloc[-1]['MA25'] and df_2.iloc[-1]['close'] > df_2.iloc[-1]['MA50']:
            # 살 가격 정하기
            # 총자산의 10퍼
            buy_price = volume / empty_count
            buy_order(buy_price, balances, bal_loc)
        else:
            print('기다리는 중')
            pass
    else:
        # 현재 보유 코인 수익률 계산
        buy_profit = ((now_price - avg_price)/avg_price) * 100
        profit = round(buy_profit, 2)

        if avg_price < sell_price or df_2.iloc[-1]['MA10'] < df_2.iloc[-1]['MA25'] or df_2.iloc[-1]['close'] < df_2.iloc[-1]['MA50']:
            sell_order(balances, bal_loc)
        else:
            print(f"코인명: {ticker}, 수익률: {profit}%")
            print("내가산 가격 = {}, 시장가격 = {}".format(avg_price, now_price))
            print('---------------------------------------------')


# 매수 주문
def buy_order(buy_price, balances, bal_loc):
    while True:
        # 8000원을 빼고 
        print("내가 산 금액 = {:,.0f}원".format(buy_price))
        print("내가 산 코인 명 : {}".format(ticker))
        balances[0]['balance'] = float(balances[0]['balance']) - buy_price
        balances[bal_loc]['currency'] = ticker
        # 8000원을 종가로 나누어 코인갯수 구하기
        balances[bal_loc]['balance'] = float(buy_price / df.iloc[-1]['close'])
        #샀을때의 가격
        balances[bal_loc]['avg_buy_price'] = float(df.iloc[-1]['close'])
        print('매수 체결')
        print('---------------------------------------------')
        return

# 매도 주문
def sell_order(balances, bal_loc):
    while True:
        # 내가 가지고 있는 코인 갯수를 종가와 곱해서 팔았을때의 가격 측정
        price_cal = df.iloc[-1]['close'] * (balances[bal_loc]['balance'])
        print("내가 판 금액 = {:,.0f}원".format(price_cal))
        print("내가 판 코인 명 : {}".format(ticker))
        balances[bal_loc]['currency'] = ''
        balances[bal_loc]['balance'] = ''
        balances[bal_loc]['avg_buy_price'] = ''
        # 팔았을 때의 가격을 기존의 지갑잔고에 더하기
        balances[0]['balance'] = balances[0]['balance'] + price_cal
        print('매도 체결')
        print('*** 내 지갑 잔고 = {:,.0f}원 ***'.format(balances[0]['balance']))
        print('---------------------------------------------')
        return

binance = login.login()
i = 0
tickers = []
ma10 = []
ma25 = []
ma50 = []
balances = []
# 시작 날짜 입력
day = datetime.datetime(2022,1,1)
day_2 = day
# 주식차트 기준 시간 입력
interval = '1h'
interval_2 = '1d'
# day = 24, minute240 = 4
time_select = 1
time_select_2 = 24
# 초기 지갑 잔고 입력
volume = 10000000

balances = ({'currency': 'dolor', 'balance': '{}'.format(volume), 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''} #1
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''} # 5
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''} # 10 실제 딕셔너리 주소
, {'currency': '', 'balance': '', 'avg_buy_price': ''})
empty_count = 10
while True:
    assets = 0
    day = int(time.mktime(day.timetuple())*1000)
    #ticker -> tickers로 변경
    for k in range(1,11):
        if balances[k]['currency'] == '':
            start_filter = True
    if start_filter == True:
        tickers = filter.filter_1(day, interval)
        tickers = filter.filter_2(tickers)
        tickers = filter.filter_3(tickers)
    start_filter = False
    # interval에 종목시간 입력
    df_2 = binance.fetch_ohlcv(symbol = 'BTC/USDT', since = day, limit= 50, timeframe = interval_2)
    df_2 = pd.DataFrame(df_2, columns = ['date', 'open', 'high', 'low', 'close', 'volume'])
    
    ma10 = df_2['close'].rolling(window=10).mean()
    df_2.insert(len(df_2.columns), "MA10", ma10)

    ma25 = df_2['close'].rolling(window=25).mean()
    df_2.insert(len(df_2.columns), "MA25", ma25)

    ma50 = df_2['close'].rolling(window=50).mean()
    df_2.insert(len(df_2.columns), "MA50", ma50)
    
    volume = float(balances[0]['balance'])
    for j in range(len(tickers)):
        ticker = tickers[j]
        df = binance.fetch_ohlcv(symbol = ticker, since = day, limit= 40, timeframe = interval)
        df = pd.DataFrame(df, columns = ['date', 'open', 'high', 'low', 'close', 'volume'])
        atr_value = atr.ATR(day, interval, ticker)
        get_ticker_ma(volume, df, balances, df_2, ticker, atr_value)
        now_price = df.iloc[-1]['close']                     # 코인 현재가
        coin_check = get_balance_coin(balances, ticker) # 코인 보유 하고 있는지 체크
        balance = coin_check[0][1]                             # 코인 보유 개수
        assets += now_price * balance
    empty_count = 0
    for z in range(1,11):
        ticker = balances[z]['currency']
        if balances[z]['currency'] == '':
            empty_count += 1
            pass
    assets = assets + balances[0]['balance']

    print("총 평가 자산 = {:,.0f}원".format(assets))    
    day = datetime.datetime.fromtimestamp(int(day)/1000)
    # 시간에 24시간씩 계속 추가
    print('---------------------------------------------')
    print(day)
    day = day + datetime.timedelta(hours = time_select)
    if i == 24:
        i = 0
        day_2 = day_2 + datetime.timedelta(hours = time_select_2)
    if day == datetime.datetime(2022, 6, 1):
        break
    i += 1
    time.sleep(0.1)


지갑 위치 = 1
내가 산 금액 = 1,000,000원
내가 산 코인 명 : RAY/USDT
매수 체결
---------------------------------------------
지갑 위치 = 2
내가 산 금액 = 1,000,000원
내가 산 코인 명 : GAL/USDT
매수 체결
---------------------------------------------
지갑 위치 = 3
내가 산 금액 = 1,000,000원
내가 산 코인 명 : ALICE/USDT
매수 체결
---------------------------------------------
지갑 위치 = 4
내가 산 금액 = 1,000,000원
내가 산 코인 명 : MASK/USDT
매수 체결
---------------------------------------------
지갑 위치 = 5
내가 산 금액 = 1,000,000원
내가 산 코인 명 : DYDX/USDT
매수 체결
---------------------------------------------
지갑 위치 = 6
내가 산 금액 = 1,000,000원
내가 산 코인 명 : EGLD/USDT
매수 체결
---------------------------------------------
지갑 위치 = 7
내가 산 금액 = 1,000,000원
내가 산 코인 명 : CELO/USDT
매수 체결
---------------------------------------------
지갑 위치 = 8
내가 산 금액 = 1,000,000원
내가 산 코인 명 : KSM/USDT
매수 체결
---------------------------------------------
지갑 위치 = 9
내가 산 금액 = 1,000,000원
내가 산 코인 명 : FLOW/USDT
매수 체결
---------------------------------------------
지갑 위치 = 10
내가 산 금액 = 1,000,000원
내가 산 코인 명 : FIL/USDT
매수 체

UnboundLocalError: local variable 'value' referenced before assignment

In [8]:
balances = ({'currency': 'dolor', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''} #1
, {'currency': '', 'balance': '', 'avg_buy_price': ''})
for k in range(1,3):
    if balances[k]['currency'] == '':
        print('진행시켜 {}번째'.format(k))
    else:
        print('꽉찼어 {}번째'.format(k))

진행시켜 1번째
진행시켜 2번째


In [8]:
def find(x):
    if x == 0:
        value = (float(), float())
        bal_loc = 0
    else:
        value = (int(), int())
        bal_loc = 1
    return value, bal_loc
sdf = find(0)
type(sdf[0][0])

float

In [18]:
balances = ({'currency': 'dolor', 'balance': '', 'avg_buy_price': ''}
, {'currency': 'BTC', 'balance': '', 'avg_buy_price': ''} #1
, {'currency': 'ETH', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''} # 5
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''}
, {'currency': '', 'balance': '', 'avg_buy_price': ''} # 10 실제 딕셔너리 주소
, {'currency': '', 'balance': '', 'avg_buy_price': ''})
balances[3]['currency']

''

In [20]:
print(type(ticker))

<class 'str'>


In [3]:

# 매게변수 ticker 추가 
def get_balance_coin(balances, ticker):
    bool = True
    while bool:
        # for문 추가
        for j in range(1, 11):
            if balances[j]['currency'] == '':
                balance = balances[j]['balance']
                avg_buy_price = balances[i]['avg_buy_price']
                value = (float(avg_buy_price), float(balance))
                bal_loc = float(i)
                bool = False
        
        value = (int(0), int(0))
        bal_loc = int(i)
        bool = False
    return value, bal_loc


In [20]:
bool = True
while True:
    for i in range(1, 11):
        if 'ETH' in balances[i]['currency']:
            bal_loc = int(i)
            bool = False
            break
    if bool == False:
        break
    for j in range(1, 11):
        if balances[j]['currency'] == '':
            a = balances[j]['currency'] 
            value = (int(0), int(0))
            bal_loc = int(j)
            print(bal_loc)
            break
    break
print("최종", bal_loc)

최종 2


In [ ]:

day = datetime.datetime(2021,1,1)
day = int(time.mktime(day.timetuple())*1000)

tickers = filter.filter_1(day, interval)
tickers = filter.filter_2(tickers)
tickers = filter.filter_3(tickers)


In [17]:
day = datetime.datetime(2021,1,1)
day = int(time.mktime(day.timetuple())*1000)
markets = binance.load_markets()
markets = pd.DataFrame(markets)
markets = markets.drop(['BTCSTUSDT', 'BTCUSDT_220930', 'ETHUSDT_220930', 'UNFI/USDT'],axis=1)
markets = markets.reset_index(drop=True)
tickers = []
tickers_5 = []
for i in range(len(markets.columns)):
    if "USDT" in markets.iloc[1][i]:
        ticker = markets.iloc[1][i]
        tickers.append(ticker)
for i in range(len(tickers)):
    ticker = tickers[i]
    ticker_now_price = binance.fetch_ohlcv(symbol = ticker, since = day, limit= 1, timeframe = interval)
    ticker_now_price = pd.DataFrame(ticker_now_price, columns = ['date', 'open', 'high', 'low', 'close', 'volume'])
    ticker_price = ticker_now_price.iloc[0]['close']
    if ticker_price > 5:
        tickers_5.append(ticker)
print(tickers_5)
        

['BTC/USDT', 'ETH/USDT', 'BCH/USDT', 'LTC/USDT', 'ETC/USDT', 'LINK/USDT', 'XMR/USDT', 'DASH/USDT', 'ZEC/USDT', 'BNB/USDT', 'ATOM/USDT', 'NEO/USDT', 'COMP/USDT', 'BAND/USDT', 'WAVES/USDT', 'MKR/USDT', 'SNX/USDT', 'DOT/USDT', 'DEFI/USDT', 'YFI/USDT', 'BAL/USDT', 'TRB/USDT', 'EGLD/USDT', 'KSM/USDT', 'AAVE/USDT', 'FIL/USDT', 'ZEN/USDT', 'LIT/USDT', 'ALICE/USDT', 'MTL/USDT', 'ICP/USDT', 'BAKE/USDT', 'GTC/USDT', 'BTCDOM/USDT', 'RAY/USDT', 'MASK/USDT', 'DYDX/USDT', 'CELO/USDT', 'AR/USDT', 'LPT/USDT', 'ENS/USDT', 'ANT/USDT', 'FLOW/USDT', 'API3/USDT', 'APE/USDT', 'BNX/USDT', 'FTT/USDT', 'GAL/USDT', 'FOOTBALL/USDT']
